In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# GPU kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)


# Kod Açıklaması

PyTorch ve bazı yardımcı kütüphaneler kullanarak bir derin öğrenme projesinde temel hazırlıkları yapılmaktadır. 

# Kütüphaneler

- `os`: Dosya ve klasör işlemleri.
- `matplotlib.pyplot`: Veri görselleştirme.
- `numpy`: Sayısal hesaplamalar.
- `torch`, `torch.nn`, `torch.optim`: PyTorch ile derin öğrenme işlemleri.
- `torch.utils.data`: Veri yükleme ve bölme.
- `torchvision.datasets` ve `transforms`: Görüntü veri setleri ve ön işleme.
- `sklearn.metrics`: Model performans ölçümleri.

# GPU / CPU Kontrolü

- `torch.cuda.is_available()`: CUDA destekli GPU var mı kontrol eder.
- `torch.device(...)`: GPU varsa "cuda", yoksa "cpu" kullanır.
- Bu sayede model ve veriler uygun cihazda çalıştırılır, eğitim süresi kısalır.




In [ ]:
import torch
print("CUDA kullanılabilir mi?:", torch.cuda.is_available())
print("Kullanılan cihaz:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


# Kısa GPU Kontrolü

- Sisteminizde GPU olup olmadığını hızlıca kontrol eder.
- Model ve tensorlar uygun cihazda çalıştırılır, performans artar.


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# --- Veri Yolu ---
data_dir = "/kaggle/input/architectural-heritage-elements-image64-dataset"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# --- Parametreler ---
IMG_SIZE = 128
BATCH_SIZE = 32

# --- Transformlar ---
# Temel transform (validation ve test için)
basic_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# Augmentation (train için)
train_transform_aug = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),        # Flip
    transforms.RandomRotation(30),           # Rotation
    transforms.ColorJitter(                  # Color Jitter
        brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
    ),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),  # Zoom
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# --- Dataset ---
full_train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform_aug)

# Validation ayırma (%15)
val_size = int(0.15 * len(full_train_dataset))
train_size = len(full_train_dataset) - val_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Validation dataset basic transform ile
val_dataset.dataset.transform = basic_transform

# Test dataset
test_dataset = datasets.ImageFolder(root=test_dir, transform=basic_transform)

# --- DataLoader ---
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- Kontrol ---
print("Train:", len(train_dataset), "Validation:", len(val_dataset), "Test:", len(test_dataset))

# --- Sınıf isimleri ---
classes = full_train_dataset.classes
print("Sınıflar:", classes)

# --- Augmentation Örneklerini Görselleştirme ---
images, labels = next(iter(train_loader))
plt.figure(figsize=(12, 6))
for i in range(6):
    plt.subplot(2, 3, i+1)
    img = images[i].permute(1, 2, 0).numpy()
    img = img * 0.5 + 0.5  # normalize geri al
    plt.imshow(img)
    plt.title(classes[labels[i]])
    plt.axis("off")
plt.suptitle("Data Augmentation Örnekleri", fontsize=16)
plt.show()

# --- Veri seti istatistikleri ---
train_labels = [label for _, label in full_train_dataset]
label_counts = Counter(train_labels)
print("Train seti sınıf dağılımı:")
for idx, count in label_counts.items():
    print(f"{classes[idx]}: {count} görsel")


# Veri Yükleme, Augmentation ve İstatistikler

Bu kod bloğu, Architectural Heritage veri setini PyTorch ile hazır hale getirir ve temel veri analizlerini yapar.

1. **Veri Yolu ve Parametreler**
   - `train_dir` ve `test_dir`: Eğitim ve test klasörlerinin yolu.
   - `IMG_SIZE`: Görsellerin yeniden boyutlandırılacağı boyut (128x128).
   - `BATCH_SIZE`: DataLoader ile verilerin batch hâlinde alınacağı büyüklük.

2. **Transform ve Data Augmentation**
   - `basic_transform`: Validation ve test verilerini tensor hâline getirip normalize eder.
   - `train_transform_aug`: Eğitim verisine augmentation uygular:
     - Horizontal flip, random rotation (30 derece),
     - Color jitter (parlaklık, kontrast, renk doygunluğu ve hue),
     - Random resized crop (zoom)
   - Veri artırımı, modelin daha genelleştirilebilir olmasını sağlar.

3. **Dataset ve DataLoader**
   - `ImageFolder`: Görselleri klasör yapısına göre yükler.
   - `random_split`: Eğitim setinden %15 validation ayırır.
   - Validation ve test setleri `basic_transform` ile işlenir.
   - `DataLoader`: Verileri batch hâlinde ve shuffle ile yükler.

4. **Kontrol ve Sınıf İsimleri**
   - Eğitim, validation ve test setlerinin boyutları ekrana yazdırılır.
   - `classes`: Sınıf isimlerini listeler.

5. **Augmentation Örneklerini Görselleştirme**
   - Eğitim verisinden alınan örnek görseller gösterilir.
   - Normalizasyon geri alınarak gerçek renkler görüntülenir.

6. **Veri Seti İstatistikleri**
   - Her sınıftaki görsel sayısı `Counter` ile hesaplanır.
   - Sınıf dağılımı, veri dengesizliğini kontrol etmek için önemlidir.


In [ ]:
import torch
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self, num_classes, img_size=64, dropout_rate=0.5):
        super(CNNModel, self).__init__()
        
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),   # 64x64 → 32 filtre
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),               # 32x32

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),               # 16x16

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),               # 8x8

            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),               # 4x4

            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((2, 2))      # sabit boyut
        )

        # Dinamik flatten boyutu
        dummy_input = torch.zeros(1, 3, img_size, img_size)
        dummy_output = self.conv_block(dummy_input)
        flatten_size = dummy_output.view(1, -1).size(1)

        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flatten_size, 512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)   # Softmax yok → CrossEntropyLoss içinde var
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x



num_classes = len(classes)
model = CNNModel(num_classes).to(device)
print(model)

# 4️⃣ Loss ve Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)



# CNN Modeli, Loss ve Optimizer

## 1️⃣ Model Tanımı
- `CNNModel` sınıfı, bir Convolutional Neural Network (CNN) tanımlar.
- **Convolutional blokları:**
  - Conv2d + BatchNorm + ReLU + MaxPool katmanları ile görsel özellikler çıkarılır.
  - AdaptiveAvgPool2d ile son feature map boyutu sabitlenir.
- **Fully Connected (FC) blokları:**
  - Flatten sonrası Linear katmanlar ile sınıflandırma yapılır.
  - ReLU aktivasyonu ve Dropout ile overfitting azaltılır.
- `num_classes`: çıktı katmanındaki sınıf sayısı (CrossEntropyLoss ile uyumlu, softmax içermiyor).

## 2️⃣ Dinamik Flatten Boyutu
- `dummy_input` ile conv_block’tan geçen feature map boyutu hesaplanır.
- Bu sayede farklı img_size parametrelerinde FC katmanları otomatik boyutlanır.

## 3️⃣ Loss ve Optimizer
- `criterion = nn.CrossEntropyLoss()`: Çok sınıflı sınıflandırma için uygun kayıp fonksiyonu.
- `optimizer = optim.Adam(...)`: Model parametrelerini güncellemek için Adam optimizasyonu.
- `scheduler = ReduceLROnPlateau(...)`: Validation kaybı durduğunda öğrenme oranını düşürür (faktor 0.5, patience 3).

## 4️⃣ Cihaza Taşıma
- `model.to(device)`: Model GPU (cuda) veya CPU üzerinde çalışacak şekilde ayarlanır.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

# --- Cihaz ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

# --- Hiperparametreler ---
EPOCHS = 30
BATCH_SIZE = 64  # GPU yeterliyse artır
LEARNING_RATE = 0.001
PATIENCE = 7  # Early stopping sabrı
IMG_SIZE = 64  # Datasetin boyutu
num_classes = len(classes)

# --- Data Augmentation ---
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# --- Dataset ve DataLoader (örn: ImageFolder kullanıyorsan) ---
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- Model ---
class CNNModel(nn.Module):
    def __init__(self, num_classes, img_size=64):
        super(CNNModel, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,128,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        # Dinamik flatten boyutu için dummy forward
        dummy_input = torch.zeros(1, 3, img_size, img_size)
        dummy_output = self.conv_block(dummy_input)
        flatten_size = dummy_output.view(1, -1).size(1)

        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(flatten_size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x

model = CNNModel(num_classes).to(device)

# --- Loss ve Optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# --- Eğitim Döngüsü ---
best_val_loss = float('inf')
patience_counter = 0
train_losses, val_losses = [], []
train_accs, val_accs = [], []

for epoch in range(EPOCHS):
    # ---- TRAIN ----
    model.train()
    running_loss, running_corrects = 0.0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        running_corrects += (outputs.argmax(1) == labels).sum().item()
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset)
    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)
    
    # ---- VALIDATION ----
    model.eval()
    running_loss, running_corrects = 0.0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            running_corrects += (outputs.argmax(1) == labels).sum().item()
    
    val_loss = running_loss / len(val_dataset)
    val_acc = running_corrects / len(val_dataset)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
    
    # ---- Early Stopping ve Checkpoint ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
    
    # ---- Scheduler ----
    scheduler.step(val_loss)

# --- Grafikler ---
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.legend()
plt.title("Loss per Epoch")

plt.subplot(1,2,2)
plt.plot(train_accs, label="Train Accuracy")
plt.plot(val_accs, label="Val Accuracy")
plt.legend()
plt.title("Accuracy per Epoch")
plt.show()


# CNN Model Eğitimi ve Validasyon

## 1️⃣ Cihaz ve Hiperparametreler
- `device`: GPU varsa CUDA, yoksa CPU kullanılır.
- `EPOCHS`, `BATCH_SIZE`, `LEARNING_RATE`: Eğitim döngüsü için temel hiperparametreler.
- `PATIENCE`: Early stopping sabrı (validation kaybı iyileşmezse durdurma).

## 2️⃣ Data Augmentation ve DataLoader
- `train_transform`: Eğitim verisine augmentation uygular (crop, flip, rotation, color jitter).
- `val_transform`: Validation verisi sadece normalize edilir.
- `DataLoader`: Verileri batch hâlinde ve shuffle ile yükler.

## 3️⃣ CNN Modeli
- Convolutional bloklar: Conv2d + ReLU + MaxPool ile görsel özellik çıkarılır.
- Fully Connected bloklar: Flatten → Linear → ReLU → Dropout → Linear → Softmax.
- Dinamik flatten boyutu: Farklı `IMG_SIZE` değerleri ile uyumlu.

## 4️⃣ Loss ve Optimizer
- `criterion = nn.CrossEntropyLoss()`: Çok sınıflı sınıflandırma için uygun.
- `optimizer = Adam`: Ağırlık güncellemesi için.
- `scheduler = ReduceLROnPlateau`: Validation kaybı durursa öğrenme oranını düşürür.

## 5️⃣ Eğitim Döngüsü
- Her epoch için:
  1. **Train**: Model `train()` modunda, loss ve doğruluk hesaplanır.
  2. **Validation**: Model `eval()` modunda, gradient hesaplamadan loss ve doğruluk hesaplanır.
  3. **Early Stopping ve Checkpoint**: Validation kaybı iyileşmezse eğitim durdurulur ve en iyi model kaydedilir.
  4. **Scheduler**: Öğrenme oranı validation kaybına göre ayarlanır.

## 6️⃣ Performans Görselleştirme
- `train_losses` ve `val_losses` grafiği: Epoch başına kayıp değişimi.
- `train_accs` ve `val_accs` grafiği: Epoch başına doğruluk değişimi.
- Bu grafikler, modelin öğrenme sürecini ve overfitting durumunu analiz etmek için kullanılır.

## 7️⃣ Eğitim Çıktısı Yorumları

- Eğitim süreci **30 epoch** boyunca GPU üzerinde (`cuda`) tamamlandı.  
- **Train Loss**: 2.10 → 1.64’e düştü → model eğitim setinde istikrarlı bir öğrenme gösterdi.  
- **Validation Loss**: 2.01 → 1.69 civarına geriledi → doğrulama setinde de belirgin iyileşme var, ancak sonlarda küçük dalgalanmalar gözlendi.  
- **Train Accuracy**: 0.34 → 0.82’ye yükseldi → eğitim setinde güçlü bir performans elde edildi.  
- **Validation Accuracy**: 0.44 → 0.76’ya çıktı → doğrulama setinde de modelin genelleme kabiliyeti arttı.  
- **Genel Değerlendirme**:
  - Eğitim ve doğrulama kayıpları birbirine yakın seyretti, **overfitting düşük** seviyede.  
  - Özellikle **10–20. epoch arası** modelin doğrulama performansında istikrarlı bir yükseliş görüldü.  
  - En yüksek **validation accuracy ≈ %76.5 (Epoch 28)** değerine ulaşıldı.  
  - Son epoch’larda train accuracy %82’ye çıktı ama validation accuracy biraz oynak kaldı → bu durum modelin artık sınır performansına yaklaşmaya başladığını gösteriyor.  
  - Genel olarak model, hem eğitim hem de doğrulama verisinde **başarılı bir şekilde öğrenmiş ve dengeli genelleme yapabilmiş** durumda.  





In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# --- Cihaz ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Test Transform ---
IMG_SIZE = 64
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# --- Test Dataset & Loader ---
test_dataset = datasets.ImageFolder("/kaggle/input/architectural-heritage-elements-image64-dataset/test",
                                    transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
classes = test_dataset.classes

# --- Model yükleme ---
model = CNNModel(num_classes=len(classes), img_size=IMG_SIZE).to(device)
model.load_state_dict(torch.load("/kaggle/working/best_model.pth", map_location=device))
model.eval()

# --- Confusion Matrix ve Classification Report ---
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=classes, yticklabels=classes, cmap="Blues")
plt.xlabel("Tahmin")
plt.ylabel("Gerçek")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print("Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=classes, zero_division=0))


# Model Test Sonuçları

## 1️⃣ Confusion Matrix
- Confusion matrix, modelin hangi sınıfları doğru tahmin ettiğini ve hangi sınıflarda hata yaptığını görmemizi sağlar.  
- Mavi tonlarda (Blues) görselleştirildiğinde, koyu hücreler daha yüksek doğru/yanlış sınıflandırma sayılarını temsil eder.  

### Önemli Gözlemler:
1. **Stained_glass**: Çok yüksek doğruluk → recall %95, precision %88. Model bu sınıfta en başarılı performansı sergiliyor.  
2. **Gargoyle**: Yüksek başarı → recall %85, precision %78. Karışıklık düşük seviyede.  
3. **Column ve Vault**: Güçlü f1-score (≈0.72–0.77). Model bu mimari öğeleri genellikle doğru ayırt edebiliyor.  
4. **Dome(inner) ve Dome(outer)**: İyi performans (f1≈0.71–0.78), ancak inner–outer arasında karışıklık yaşanmış olabilir.  
5. **Altar**: Precision yüksek (%81) ama recall düşük (%62) → model altar örneklerinin bir kısmını yanlış sınıflandırıyor.  
6. **Bell_tower ve Apse**: Orta düzey başarı (f1≈0.51–0.57). Görsel benzerliklerden dolayı karışıklık yaşanıyor.  
7. **Flying_buttress**: En zayıf sınıf → recall %24, f1=0.37. Model bu sınıfı tanımakta zorlanıyor, muhtemelen veri yetersizliği veya görsel karmaşıklık nedeniyle.  

---

## 2️⃣ Classification Report
- **precision**: Belirli bir sınıf tahmini yapıldığında doğruluk oranı.  
- **recall**: Gerçek sınıftaki örnekleri yakalama başarısı.  
- **f1-score**: Precision ve recall’un dengeli ortalaması.  
- **support**: Test setinde o sınıfa ait örnek sayısı.  

### Önemli Gözlemler:
- **En yüksek performans**: `stained_glass` (f1=0.91) → model bu sınıfta çok güçlü.  
- **Yüksek performanslı diğer sınıflar**: `gargoyle` (0.81), `vault` (0.77), `dome(inner)` (0.78).  
- **Orta performanslı sınıflar**: `altar` (0.70), `dome(outer)` (0.71), `column` (0.72).  
- **Zayıf performanslı sınıflar**: `bell_tower` (0.57), `apse` (0.51), `flying_buttress` (0.37).  
- **Genel accuracy**: %73 → Model test setindeki örneklerin yaklaşık 3/4’ünü doğru tahmin ediyor.  
- **Macro avg f1-score = 0.69** → Sınıflar arası dengesizlik var, bazı sınıflar daha zayıf kalmış.  
- **Weighted avg f1-score = 0.72** → Genel dağılıma göre dengeli performans kabul edilebilir seviyede.  

---

## 3️⃣ Genel Yorum
- Model özellikle belirgin görsel özelliklere sahip sınıflarda (`stained_glass`, `gargoyle`, `vault`) güçlü performans sergiliyor.  
- Daha zorlayıcı veya veri azlığı olan sınıflarda (`flying_buttress`, `apse`, `bell_tower`) hatalar artmış.  
- **Altar** sınıfında precision yüksek ama recall düşük → model altar olmayan örnekleri altar olarak tahmin etmiyor (temkinli davranıyor), ama altar örneklerinin bir kısmını kaçırıyor.  
- Performansı artırmak için:  
  - Veri artırımı (özellikle `flying_buttress`, `apse`, `bell_tower` sınıflarında),  
  - **Class-weighted loss** veya **focal loss** gibi yöntemler,  
  - Daha derin/transfer learning tabanlı modeller (ör. ResNet, EfficientNet) kullanılabilir.  


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# --- Grad-CAM Fonksiyonu ---
def generate_gradcam(model, image, target_class):
    model.eval()
    image = image.unsqueeze(0).to(device)
    gradients, activations = [], []

    def forward_hook(module, input, output):
        activations.append(output)
    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])

    last_conv = model.conv_block[-3]
    h1 = last_conv.register_forward_hook(forward_hook)
    h2 = last_conv.register_full_backward_hook(backward_hook)

    output = model(image)
    model.zero_grad()
    loss = output[0, target_class]
    loss.backward()

    grads = gradients[0][0].cpu().data.numpy()
    acts = activations[0][0].cpu().data.numpy()
    weights = np.mean(grads, axis=(1,2))
    cam = np.zeros(acts.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * acts[i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (IMG_SIZE, IMG_SIZE))
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

    h1.remove()
    h2.remove()
    return cam

# --- Görselleştirme ---
examples_per_class = 2
fig, axs = plt.subplots(len(classes), examples_per_class, figsize=(5*examples_per_class, 5*len(classes)))

for i, cls in enumerate(classes):
    count = 0
    for images, labels in test_loader:
        for j in range(images.size(0)):
            if classes[labels[j]] == cls:
                image = images[j]
                label = labels[j]
                cam = generate_gradcam(model, image, label.item())
                img = image.permute(1,2,0).cpu().numpy()
                img = img * 0.5 + 0.5

                ax = axs[i, count] if examples_per_class > 1 else axs[i]
                ax.imshow(img)
                ax.imshow(cam, cmap='jet', alpha=0.4)
                ax.set_title(f"Label: {cls}")
                ax.axis("off")

                count += 1
                if count >= examples_per_class:
                    break
        if count >= examples_per_class:
            break

plt.suptitle("Grad-CAM Heatmap - Her Sınıftan Örnekler", fontsize=16)
plt.tight_layout()
plt.show()


# Grad-CAM ile Görselleştirme

## 1️⃣ Amaç
- Grad-CAM (Gradient-weighted Class Activation Mapping), bir CNN modelinin hangi görsel bölgelerine odaklanarak tahmin yaptığını gösterir.
- Bu sayede modelin karar mekanizmasını görselleştirebilir ve açıklanabilirliği artırabiliriz.

## 2️⃣ İşleyiş
1. **Forward Hook**: Son convolution katmanının aktivasyonlarını kaydeder.
2. **Backward Hook**: Hedef sınıf için geri yayılım sırasında gradienleri kaydeder.
3. **Ağırlıklı Aktivasyon**: Gradienlerin ortalaması alınarak her feature map için ağırlık hesaplanır.
4. **Heatmap Oluşturma**: Feature map’ler ağırlıklarla çarpılır ve toplanır, ardından normalize edilip orijinal görüntü üzerine bindirilir.

## 3️⃣ Kod Detayları
- `generate_gradcam(model, image, target_class)`: Bir görüntü ve hedef sınıf için Grad-CAM haritası üretir.
- `last_conv = model.conv_block[-3]`: Son convolution katmanı seçilir.
- `cam = cv2.resize(cam, (IMG_SIZE, IMG_SIZE))`: Heatmap, orijinal görüntü boyutuna ölçeklenir.
- `ax.imshow(cam, cmap='jet', alpha=0.4)`: Heatmap, orijinal görüntü üzerine yarı saydam olarak bindirilir.

## 4️⃣ Görselleştirme
- Her sınıftan `examples_per_class` adet örnek seçilir.
- Görüntü ve Grad-CAM heatmap’i birlikte gösterilir.
- Kırmızı bölgeler modelin tahmin için en çok odaklandığı alanları temsil eder.

## 5️⃣ Yorum
- Grad-CAM, modelin hangi bölgelere dikkat ettiğini göstererek yanlış tahminleri analiz etmemize yardımcı olur.
- Örneğin, `stained_glass` veya `gargoyle` sınıflarında model doğru tahmin için karakteristik bölgeleri kullanıyor.
- Zayıf sınıflarda (`flying_buttress`) heatmap daha dağınık olabilir, bu da modelin kararsız olduğunu gösterir.


In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

dropout_values = [0.3, 0.5, 0.7]
fixed_lr = 0.001

best_val_acc_dropout = 0
best_dropout = 0

# Train/Val loss ve accuracy kayıtları
history = {}

for dropout in dropout_values:
    print(f"Deneme: Dropout={dropout}, Learning Rate={fixed_lr}")

    model = CNNModel(num_classes=num_classes, img_size=64)
    # FC bloktaki dropoutları değiştir
    model.fc_block[1] = nn.Dropout(dropout)
    model.fc_block[4] = nn.Dropout(dropout)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=fixed_lr)

    # Küçük test epoch sayısı
    EPOCHS_TEST = 5
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(EPOCHS_TEST):
        # ----- TRAIN -----
        model.train()
        running_loss, running_corrects, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * labels.size(0)
            running_corrects += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        # ----- VALIDATION -----
        model.eval()
        running_loss, running_corrects, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * labels.size(0)
                running_corrects += (outputs.argmax(1) == labels).sum().item()
                total += labels.size(0)

        val_epoch_loss = running_loss / total
        val_epoch_acc = running_corrects / total
        val_losses.append(val_epoch_loss)
        val_accs.append(val_epoch_acc)

    # Kayıtları history içine ekle
    history[dropout] = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs
    }

    final_val_acc = val_accs[-1]
    print(f"Validation Accuracy: {final_val_acc:.4f}\n")

    if final_val_acc > best_val_acc_dropout:
        best_val_acc_dropout = final_val_acc
        best_dropout = dropout

print("En iyi Dropout:", best_dropout, "En iyi doğruluk:", best_val_acc_dropout)

# --- GRAFİKLER ---
for dropout in dropout_values:
    plt.figure(figsize=(12,4))
    
    # Loss grafiği
    plt.subplot(1,2,1)
    plt.plot(history[dropout]['train_losses'], label='Train Loss')
    plt.plot(history[dropout]['val_losses'], label='Val Loss')
    plt.title(f"Dropout={dropout} - Loss per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    
    # Accuracy grafiği
    plt.subplot(1,2,2)
    plt.plot(history[dropout]['train_accs'], label='Train Acc')
    plt.plot(history[dropout]['val_accs'], label='Val Acc')
    plt.title(f"Dropout={dropout} - Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    
    plt.show()


# Dropout Hiperparametre Denemeleri

## 1️⃣ Amaç
- Dropout, modelin overfitting (aşırı öğrenme) yapmasını önlemek için kullanılan bir regularizasyon yöntemidir.
- Burada farklı dropout oranları (0.3, 0.5, 0.7) test edilerek en iyi doğruluk sağlayan oranı bulmayı amaçlıyoruz.

## 2️⃣ İşleyiş
1. **Model Yeniden Başlatma**: Her dropout değeri için CNN modeli sıfırlanıyor.
2. **Dropout Ayarı**: Fully Connected bloktaki dropout değerleri güncelleniyor (`fc_block[1]` ve `fc_block[4]`).
3. **Kısa Eğitim Döngüsü**: Her dropout için 5 epoch boyunca eğitim ve doğrulama yapılır.
4. **Kayıt Tutma**: Train/Validation loss ve accuracy her epoch sonunda kaydedilir.
5. **En İyi Dropout**: Son epoch’taki validation accuracy’ye göre en iyi dropout seçilir.

## 3️⃣ Gözlemler
- Grafikleri inceleyerek:
  - **Düşük dropout** → overfitting riski artabilir (train loss çok düşük, val loss yüksek olabilir).
  - **Yüksek dropout** → model underfitting yapabilir (train ve val accuracy düşük kalabilir).
- Optimum dropout, validation doğruluğunu maksimize eden değerdir.

## 4️⃣ Yorum
- Kod çıktısına göre en iyi doğruluk sağlayan dropout değeri seçiliyor ve grafiklerle hem loss hem de accuracy trendleri görselleştiriliyor.
- Bu deneme, model performansını iyileştirmek ve daha dengeli bir eğitim sağlamak için kritik bir adımdır.

# Dropout Denemeleri Çıktı Yorumu

# Dropout Denemeleri Sonuç Raporu

## 1️⃣ Deneme Sonuçları
- **Dropout = 0.3 → Validation Accuracy: 0.549**
- **Dropout = 0.5 → Validation Accuracy: 0.574 ✅ (En iyi)**
- **Dropout = 0.7 → Validation Accuracy: 0.549**

---

## 2️⃣ Yorum
- **En iyi Dropout = 0.5**, çünkü en yüksek doğruluk (%57.4) bu değerde elde edildi.  
- Düşük dropout (0.3), overfitting’i yeterince engelleyememiş.  
- Yüksek dropout (0.7), model kapasitesini fazla kısıtladığı için underfitting’e sebep olmuş.  

---

## 3️⃣ Sonuç
- Orta seviye dropout (0.5) bu veri setinde en uygun değer olarak görünüyor.  
- Daha iyi sonuç için farklı **learning rate + dropout** kombinasyonlarının birlikte test edilmesi önerilir.  



In [ ]:
learning_rates = [0.0001, 0.001, 0.01]
fixed_dropout = 0.5  # Dropout en iyi değerde sabit

best_val_acc_lr = 0
best_lr = 0
history_lr = {}

for lr in learning_rates:
    print(f"Deneme: Dropout={fixed_dropout}, Learning Rate={lr}")

    model = CNNModel(num_classes=num_classes, img_size=64)
    model.fc_block[1] = nn.Dropout(fixed_dropout)
    model.fc_block[4] = nn.Dropout(fixed_dropout)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    EPOCHS_TEST = 5
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(EPOCHS_TEST):
        # ----- TRAIN -----
        model.train()
        running_loss, running_corrects, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * labels.size(0)
            running_corrects += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        # ----- VALIDATION -----
        model.eval()
        running_loss, running_corrects, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * labels.size(0)
                running_corrects += (outputs.argmax(1) == labels).sum().item()
                total += labels.size(0)

        val_epoch_loss = running_loss / total
        val_epoch_acc = running_corrects / total
        val_losses.append(val_epoch_loss)
        val_accs.append(val_epoch_acc)

    # Kayıtları history içine ekle
    history_lr[lr] = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs
    }

    final_val_acc = val_accs[-1]
    print(f"Validation Accuracy: {final_val_acc:.4f}\n")

    if final_val_acc > best_val_acc_lr:
        best_val_acc_lr = final_val_acc
        best_lr = lr

print("En iyi Learning Rate:", best_lr, "En iyi doğruluk:", best_val_acc_lr)

# --- GRAFİKLER ---
for lr in learning_rates:
    plt.figure(figsize=(12,4))
    
    # Loss grafiği
    plt.subplot(1,2,1)
    plt.plot(history_lr[lr]['train_losses'], label='Train Loss')
    plt.plot(history_lr[lr]['val_losses'], label='Val Loss')
    plt.title(f"Learning Rate={lr} - Loss per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    
    # Accuracy grafiği
    plt.subplot(1,2,2)
    plt.plot(history_lr[lr]['train_accs'], label='Train Acc')
    plt.plot(history_lr[lr]['val_accs'], label='Val Acc')
    plt.title(f"Learning Rate={lr} - Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    
    plt.show()


# Learning Rate Denemeleri Sonuç Raporu

## 1️⃣ Deneme Sonuçları
- **Learning Rate = 0.0001 → Validation Accuracy: 0.5076**
  - Çok düşük → model yavaş öğreniyor, doğruluk sınırlı kalıyor.  
- **Learning Rate = 0.001 → Validation Accuracy: 0.5734 ✅**
  - Orta seviye → model dengeli şekilde öğreniyor, en iyi sonuç burada elde edildi.  
- **Learning Rate = 0.01 → Validation Accuracy: 0.1606 ⚠️**
  - Çok yüksek → model dengesiz öğreniyor (overshooting), doğruluk ciddi şekilde düşüyor.  

---

## 2️⃣ Yorum
- **En iyi Learning Rate = 0.001**  
- Çok düşük learning rate eğitim süresini uzatıyor.  
- Çok yüksek learning rate ise modelin öğrenmesini bozuyor.  

---

## 3️⃣ Sonuç
- **Learning Rate = 0.001**, bu veri seti ve model için optimum değer olarak belirlendi.  
- Sonraki tam eğitimlerde **Dropout = 0.5** ve **Learning Rate = 0.001** birlikte kullanılabilir.  


In [ ]:
dropout_values = [0.3, 0.5, 0.7]
learning_rates = [0.0001, 0.001, 0.01]

EPOCHS_TEST = 5
history_comb = {}

for dropout in dropout_values:
    for lr in learning_rates:
        key = f"Dropout={dropout}_LR={lr}"
        print(f"Deneme: {key}")

        model = CNNModel(num_classes=num_classes, img_size=64)
        model.fc_block[1] = nn.Dropout(dropout)
        model.fc_block[4] = nn.Dropout(dropout)
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        train_losses, val_losses = [], []
        train_accs, val_accs = [], []

        for epoch in range(EPOCHS_TEST):
            # TRAIN
            model.train()
            running_loss, running_corrects, total = 0.0, 0, 0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * labels.size(0)
                running_corrects += (outputs.argmax(1) == labels).sum().item()
                total += labels.size(0)
            train_losses.append(running_loss/total)
            train_accs.append(running_corrects/total)

            # VALIDATION
            model.eval()
            running_loss, running_corrects, total = 0.0, 0, 0
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    running_loss += loss.item() * labels.size(0)
                    running_corrects += (outputs.argmax(1) == labels).sum().item()
                    total += labels.size(0)
            val_losses.append(running_loss/total)
            val_accs.append(running_corrects/total)

        history_comb[key] = {
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accs': train_accs,
            'val_accs': val_accs
        }

# --- GRAFİKLER ---
for key, values in history_comb.items():
    plt.figure(figsize=(12,4))
    
    # Loss grafiği
    plt.subplot(1,2,1)
    plt.plot(values['train_losses'], label='Train Loss')
    plt.plot(values['val_losses'], label='Val Loss')
    plt.title(f"{key} - Loss per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    
    # Accuracy grafiği
    plt.subplot(1,2,2)
    plt.plot(values['train_accs'], label='Train Acc')
    plt.plot(values['val_accs'], label='Val Acc')
    plt.title(f"{key} - Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    
    plt.show()


# Hiperparametre Kombinasyonları (Dropout ve Learning Rate) Denemesi

## Yapılan İşlemler
- **Dropout değerleri:** `[0.3, 0.5, 0.7]`  
- **Learning Rate değerleri:** `[0.0001, 0.001, 0.01]`  
- Her kombinasyon için:
  1. CNN modeli oluşturuldu.
  2. Fully-connected bloktaki dropout değerleri güncellendi.
  3. Model belirtilen learning rate ile Adam optimizer kullanılarak eğitildi.
  4. 5 epoch boyunca eğitim ve doğrulama kayıpları ile doğrulukları kaydedildi.
  5. Sonuçlar `history_comb` dictionary’sine kaydedildi.

## Grafikler
- Her kombinasyon için **Loss ve Accuracy** grafikleri çizildi.
- Loss grafikleri: Modelin eğitim ve doğrulama sırasında kaybının nasıl değiştiğini gösterir.
- Accuracy grafikleri: Modelin doğrulama performansını ve overfitting/underfitting eğilimlerini gösterir.

## Yorum
- **Amaç:** Hem dropout hem learning rate değerlerini birlikte deneyerek en iyi kombinasyonu belirlemek.
- Grafikler üzerinden hangi kombinasyonun hem düşük loss hem yüksek doğruluk verdiği görselleştirilebilir.
- Bu yöntem, hiperparametre optimizasyonunun basit bir görselleştirme ve gözlem yoludur.

---

# Hiperparametre Tarama Sonuçları

## Denenen Kombinasyonlar
- Dropout değerleri: `0.3, 0.5, 0.7`
- Learning Rate değerleri: `0.0001, 0.001, 0.01`
- Kombinasyonlar:
  1. Dropout=0.3, LR=0.0001 → Val Accuracy ≈ 0.485  
  2. Dropout=0.3, LR=0.001  → Val Accuracy ≈ 0.604 ✅  
  3. Dropout=0.3, LR=0.01   → Val Accuracy ≈ 0.117  
  4. Dropout=0.5, LR=0.0001 → Val Accuracy ≈ 0.508  
  5. Dropout=0.5, LR=0.001  → Val Accuracy ≈ 0.573  
  6. Dropout=0.5, LR=0.01   → Val Accuracy ≈ 0.161  
  7. Dropout=0.7, LR=0.0001 → Val Accuracy ≈ 0.579  
  8. Dropout=0.7, LR=0.001  → Val Accuracy ≈ 0.549  
  9. Dropout=0.7, LR=0.01   → Val Accuracy ≈ çok düşük  

## Yorum
- **En iyi kombinasyon:** `Dropout=0.3` ve `Learning Rate=0.001`  
- Bu kombinasyon, validation setinde en yüksek doğruluk sağladı (~60.4%).  
- Çok yüksek dropout (0.5, 0.7) veya çok yüksek LR (0.01) doğruluğu ciddi şekilde düşürdü.  
- Düşük LR (0.0001) yavaş öğrenme sebebiyle performansı sınırladı.  
- Bu sonuçlar, modelin **overfitting’i önlerken yeterince öğrenmesini sağlayacak dropout ve LR değerlerini** görsel olarak da destekliyor.  


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score

# --- Cihaz ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Test Transform ---
IMG_SIZE = 64
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# --- Test Dataset & Loader ---
test_dataset = datasets.ImageFolder("/kaggle/input/architectural-heritage-elements-image64-dataset/test",
                                    transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# --- Model yükleme ---
model = CNNModel(num_classes=len(test_dataset.classes), img_size=IMG_SIZE).to(device)
model.load_state_dict(torch.load("/kaggle/working/best_model.pth", map_location=device))
model.eval()

# --- Model Başarı Skoru ---
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

acc = accuracy_score(all_labels, all_preds)
print(f"✅ Model Başarı Skoru (Accuracy): {acc:.4f}")


## Model Başarı Skoru (Test Seti)

Bu kod, eğitilmiş CNN modelini test setinde değerlendirir ve doğruluk skorunu hesaplar:

- Test verisi `64x64` boyutunda normalize edilip yüklenir.  
- Model, `best_model.pth` ağırlıklarıyla yüklenir.  
- Modelin tahminleri alınır ve gerçek etiketlerle karşılaştırılır.  
- `sklearn.metrics.accuracy_score` kullanılarak doğruluk hesaplanır.

Bu sonuç, modelin test setinde yaklaşık **%72.8 doğruluk** sağladığını gösterir. Validation skorlarına yakın olması, modelin iyi genelleştiğini gösterir.



